In [38]:
import torch

In [39]:
with open('divina_commedia.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()


In [40]:
print(text[:200])

LA DIVINA COMMEDIA
di Dante Alighieri
INFERNO



Inferno: Canto I

  Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura
ché la diritta via era smarrita.
  Ahi quanto a dir qual era è


In [41]:
len(text)

537093

In [42]:
chars = sorted(list(set(text)))
print(len(chars))
print("".join(chars))

68

 !"'(),-.:;?ABCDEFGHILMNOPQRSTUVXZabcdefghijlmnopqrstuvxyz~àèéìïòóù


In [43]:
char_to_int = { ch: i for i,ch in enumerate(chars)}
int_to_char = { i: ch for i,ch in enumerate(chars)}
encode = lambda s: [char_to_int[c] for c in s] #encoder function
decode = lambda l: "".join(int_to_char[i] for i in l) #decoder function

#not byte pair encoding, for simplicity

In [44]:
decode(encode("cianoxxj"))


'cianoxxj'

In [45]:
encode("ciao mondo")

[37, 43, 35, 48, 1, 46, 48, 47, 38, 48]

In [46]:
data = torch.tensor(encode(text))

In [47]:
n = int(0.9 * len(data))
train_data= data[:n]
val_data= data[n:]


In [52]:
batch_size = 4
block_size= 10

train_data[:block_size+1]

tensor([22, 13,  1, 16, 21, 32, 21, 24, 13,  1, 15])

In [53]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 10])
tensor([[ 7,  0, 38, 39, 45,  1, 46, 43, 48,  1],
        [39,  1, 43, 47,  1, 37, 48, 52, 53, 35],
        [50, 54, 35, 45,  1, 52, 39, 41, 54, 39],
        [46, 48, 45, 53, 39,  1, 55, 48, 45, 53]])
targets:
torch.Size([4, 10])
tensor([[ 0, 38, 39, 45,  1, 46, 43, 48,  1, 37],
        [ 1, 43, 47,  1, 37, 48, 52, 53, 35,  1],
        [54, 35, 45,  1, 52, 39, 41, 54, 39,  1],
        [48, 45, 53, 39,  1, 55, 48, 45, 53, 39]])
----
when input is [7] the target: 0
when input is [7, 0] the target: 38
when input is [7, 0, 38] the target: 39
when input is [7, 0, 38, 39] the target: 45
when input is [7, 0, 38, 39, 45] the target: 1
when input is [7, 0, 38, 39, 45, 1] the target: 46
when input is [7, 0, 38, 39, 45, 1, 46] the target: 43
when input is [7, 0, 38, 39, 45, 1, 46, 43] the target: 48
when input is [7, 0, 38, 39, 45, 1, 46, 43, 48] the target: 1
when input is [7, 0, 38, 39, 45, 1, 46, 43, 48, 1] the target: 37
when input is [39] the target: 1
when input 